In [2]:
from shapely.geometry import Point, Polygon
from matplotlib import pyplot as plt
from math import sin, cos, sqrt, atan2, radians

# !pip install gmaps
import shapely
import pickle
import random
import numpy as np
import gmaps, os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from ipywidgets.embed import embed_minimal_html
import webbrowser

# directory containing data
DATADIR = "/home/nirvan/Desktop/github/geoguessrLSTM/infoExtraction/data"
# directory to store model in
MODELDIR = "/home/nirvan/Desktop/github/GeoguessrAI/machineLearning"

# DATADIR = '/content/drive/My Drive/geoguessr'
# MODELDIR = '/content/drive/My Drive/geoguessr'

In [3]:
# load training image file names
TF = np.load(DATADIR+'/trainFiles.npy')
# load testing image file names
TESF = np.load(DATADIR+'/testFiles.npy')
print("Train files: {}, Test files: {}".format(len(TF), len(TESF)))

Train files: 8748, Test files: 972


In [10]:
X = np.array(list(map(lambda x: [np.array(load_img(DATADIR+"/dataCombined/"+x+'/'+i, 
                                                           target_size=(300,600))) 
                                         for i in os.listdir(DATADIR+"/dataCombined/"+x)], 
                              TF[0:3])))
y = tf.keras.utils.to_categorical(list(map(lambda x:int(x.split('+')[0]), TF[0:3])), 
                                  num_classes=243)

In [11]:
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Dropout
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D
from tensorflow.keras.applications.resnet50 import ResNet50
def build_convnet(shape=(300, 600, 3)):
    momentum = .9
    model = tf.keras.Sequential()
#     model.add(Conv2D(64, (3,3), input_shape=shape,
#         padding='same', activation='relu'))
#     model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
#     model.add(BatchNormalization(momentum=momentum))
    
#     model.add(MaxPool2D())
    
#     model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
#     model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
#     model.add(BatchNormalization(momentum=momentum))
    
#     model.add(MaxPool2D())
    
#     model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
#     model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
#     model.add(BatchNormalization(momentum=momentum))
    
#     model.add(MaxPool2D())
    
#     model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
#     model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
#     model.add(BatchNormalization(momentum=momentum))
    
    restnet = ResNet50(include_top=False, weights='imagenet', input_shape=shape)
    restnet.trainable = False
    model.add(restnet)
    # flatten...
    model.add(GlobalMaxPool2D())
    return model

def action_model(shape=(3, 300, 600, 3), nbout=243):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = tf.keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

In [12]:
m = action_model()
m.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 3, 2048)           23587712  
_________________________________________________________________
gru (GRU)                    (None, 64)                405888    
_________________________________________________________________
dense (Dense)                (None, 1024)              66560     
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

In [13]:
m.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer='adam', metrics=['categorical_accuracy'])

In [14]:
m.fit(X,y)

Train on 3 samples
3/3 [==============================] - 6s 2s/sample - loss: 5.5645 - categorical_accuracy: 0.0000e+00


(2, 243)